In [1]:
import pandas as pd
import json
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Model
from keras.layers import Dense,Input
from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def simpleton(data,ignore,encode_cols,labels):
    master_data={}
    master_y={}
    for key,values in data.items():
        if key not in ignore:
            if key in encode_cols:
                val=values
                #temp_data=enc.fit_transform(val)
                master_data[key]=val
            elif key in labels:
                val=values
                #temp_data=enc.fit_transform(val)
                master_y[key]=val
            else:
                if len(values)>=0:
                    for key1,values1 in values.items():
                        val=values1
                        master_data[key+key1]=values1
    return master_data,master_y

In [3]:
# with open('train.json') as f:
#     data=json.load(f)
# ignore=['location','sleepTime','id','timestamp','token','batteryLevel','username','source']
# encode_cols=['isMusic','reachability','orientation']
# labels=['usingPhone']

# enc=preprocessing.LabelEncoder()

# sample_data=[]
# sample_labels=[]
# for i in range(len(data)):
#     needed_data,needed_labels=simpleton(data[i],ignore,encode_cols,labels)
#     sample_data.append(needed_data)
#     sample_labels.append(needed_labels)
    
# master_par=pd.DataFrame(sample_data)
# master_lab=pd.DataFrame(sample_labels)

# master_par.head(5)

In [4]:
# X=[]
# Y=master_lab['usingPhone']
# for key,val in master_par.items():
#     if key in encode_cols:
#         temp_data=enc.fit_transform(val)
#         X.append(temp_data)
#     else:
#         X.append(val)
# X=np.asarray(X)
# Y=np.asarray(Y)
# X=np.moveaxis(X,0,-1)
# Y=to_categorical(Y)
# X[0]

In [6]:
data=pd.read_csv('device_collected.csv')
columns=data.columns
print(columns)

ignoring=['timestamp', 'source', 'sleepTime','id','username','token','usingPhone','batteryLevel','location.altitude','location.latitude','location.speed','location.longitude','location.floor','location.course','location.locationName','location.horizontalAccuracy','location.verticalAccuracy',]
encode_cols=['isMusic','orientation','reachability']
lab=['usingPhone']

enc=preprocessing.LabelEncoder()

master_data={}
master_y={}
for name in columns:
    if name not in ignoring:
        print(name)
        if name in encode_cols:
            val=list(data[name])
            temp_data=enc.fit_transform(val)
            master_data[name]=temp_data
        else:
            val=np.asarray(list(data[name]))
            master_data[name]=val
    elif name in lab:
        val=list(data[name])
        temp_data=enc.fit_transform(val)
        master_y[name]=temp_data

Index(['timestamp', 'source', 'sleepTime', 'isMusic', 'id', 'orientation',
       'token', 'batteryLevel', 'usingPhone', 'reachability', 'username',
       'location.course', 'location.floor', 'location.longitude',
       'location.horizontalAccuracy', 'location.locationName',
       'location.speed', 'location.verticalAccuracy', 'location.latitude',
       'location.altitude', 'rotation.m31', 'rotation.m21', 'rotation.m11',
       'rotation.m33', 'rotation.m23', 'rotation.m13', 'rotation.m32',
       'rotation.m22', 'rotation.m12', 'attitude.x', 'attitude.y',
       'attitude.z', 'rotationRate.x', 'rotationRate.y', 'rotationRate.z',
       'acceleration.x', 'acceleration.y', 'acceleration.z', 'accelerometer.x',
       'accelerometer.y', 'accelerometer.z', 'gravity.x', 'gravity.y',
       'gravity.z', 'gyroscope.x', 'gyroscope.y', 'gyroscope.z',
       'magneticField.x', 'magneticField.y', 'magneticField.z'],
      dtype='object')
isMusic
orientation
reachability
rotation.m31
rotation.

In [7]:
X=[]
Y=master_y['usingPhone']
for key,val in master_data.items():
    X.append(val)
X=np.asarray(X)
Y=np.asarray(Y)
X=np.moveaxis(X,0,-1)
Y=to_categorical(Y)
Y[0]

array([1., 0.], dtype=float32)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [12]:
x = Input(shape=(X.shape[1],))
x_1 = Dense(16, activation='relu')(x)
y = Dense(2,activation='sigmoid')(x_1)
model_1 = Model(x, y)

# model_1=tf.keras.models.Sequential([
#     tf.keras.layers.Input(shape=(X.shape[1],),name='input'),
#     tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(2, activation='softmax',name='output')
# ])
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 33)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 578
Trainable params: 578
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.fit(X_train, y_train,epochs=200, batch_size=4, verbose=1,validation_data=(X_test,y_test))
model_1.save('nudges_dev__2_new.h5')

Train on 399 samples, validate on 198 samples
Epoch 1/200
399/399 [==============================] - 0s 628us/step - loss: 0.6793 - accuracy: 0.5815 - val_loss: 0.6290 - val_accuracy: 0.5354
Epoch 2/200
399/399 [==============================] - 0s 183us/step - loss: 0.5924 - accuracy: 0.6942 - val_loss: 0.5865 - val_accuracy: 0.5808
Epoch 3/200
399/399 [==============================] - 0s 196us/step - loss: 0.5407 - accuracy: 0.7393 - val_loss: 0.5582 - val_accuracy: 0.7222
Epoch 4/200
399/399 [==============================] - 0s 183us/step - loss: 0.5016 - accuracy: 0.7619 - val_loss: 0.5280 - val_accuracy: 0.7273
Epoch 5/200
399/399 [==============================] - 0s 183us/step - loss: 0.4695 - accuracy: 0.7744 - val_loss: 0.5224 - val_accuracy: 0.7172
Epoch 6/200
399/399 [==============================] - 0s 231us/step - loss: 0.4471 - accuracy: 0.7744 - val_loss: 0.5037 - val_accuracy: 0.7222
Epoch 7/200
399/399 [==============================] - 0s 178us/step - loss: 0.4320 

Epoch 57/200
399/399 [==============================] - 0s 186us/step - loss: 0.3499 - accuracy: 0.8020 - val_loss: 0.5106 - val_accuracy: 0.7121
Epoch 58/200
399/399 [==============================] - 0s 192us/step - loss: 0.3525 - accuracy: 0.7895 - val_loss: 0.5157 - val_accuracy: 0.7273
Epoch 59/200
399/399 [==============================] - 0s 190us/step - loss: 0.3497 - accuracy: 0.8195 - val_loss: 0.5096 - val_accuracy: 0.7172
Epoch 60/200
399/399 [==============================] - 0s 192us/step - loss: 0.3497 - accuracy: 0.8045 - val_loss: 0.5171 - val_accuracy: 0.7273
Epoch 61/200
399/399 [==============================] - 0s 199us/step - loss: 0.3486 - accuracy: 0.7970 - val_loss: 0.5145 - val_accuracy: 0.7273
Epoch 62/200
399/399 [==============================] - 0s 189us/step - loss: 0.3497 - accuracy: 0.8045 - val_loss: 0.5113 - val_accuracy: 0.7172
Epoch 63/200
399/399 [==============================] - 0s 189us/step - loss: 0.3475 - accuracy: 0.8070 - val_loss: 0.5114 -

399/399 [==============================] - 0s 186us/step - loss: 0.3227 - accuracy: 0.8396 - val_loss: 0.5565 - val_accuracy: 0.7273
Epoch 114/200
399/399 [==============================] - 0s 187us/step - loss: 0.3224 - accuracy: 0.8321 - val_loss: 0.5546 - val_accuracy: 0.7374
Epoch 115/200
399/399 [==============================] - 0s 191us/step - loss: 0.3217 - accuracy: 0.8371 - val_loss: 0.5588 - val_accuracy: 0.7323
Epoch 116/200
399/399 [==============================] - 0s 189us/step - loss: 0.3207 - accuracy: 0.8346 - val_loss: 0.5503 - val_accuracy: 0.7172
Epoch 117/200
399/399 [==============================] - 0s 191us/step - loss: 0.3213 - accuracy: 0.8421 - val_loss: 0.5500 - val_accuracy: 0.7121
Epoch 118/200
399/399 [==============================] - 0s 191us/step - loss: 0.3183 - accuracy: 0.8296 - val_loss: 0.5639 - val_accuracy: 0.7323
Epoch 119/200
399/399 [==============================] - 0s 186us/step - loss: 0.3198 - accuracy: 0.8396 - val_loss: 0.5677 - val_ac

Epoch 169/200
399/399 [==============================] - 0s 366us/step - loss: 0.2965 - accuracy: 0.8622 - val_loss: 0.6427 - val_accuracy: 0.7677
Epoch 170/200
399/399 [==============================] - 0s 246us/step - loss: 0.3022 - accuracy: 0.8521 - val_loss: 0.6228 - val_accuracy: 0.7374
Epoch 171/200
399/399 [==============================] - 0s 459us/step - loss: 0.2995 - accuracy: 0.8496 - val_loss: 0.6294 - val_accuracy: 0.7525
Epoch 172/200
399/399 [==============================] - 0s 1ms/step - loss: 0.2948 - accuracy: 0.8446 - val_loss: 0.6569 - val_accuracy: 0.7677
Epoch 173/200
399/399 [==============================] - 0s 282us/step - loss: 0.2970 - accuracy: 0.8521 - val_loss: 0.6522 - val_accuracy: 0.7727
Epoch 174/200
399/399 [==============================] - 0s 248us/step - loss: 0.2965 - accuracy: 0.8647 - val_loss: 0.6525 - val_accuracy: 0.7727
Epoch 175/200
399/399 [==============================] - 0s 203us/step - loss: 0.2905 - accuracy: 0.8797 - val_loss: 0.6

In [16]:
sam=[0,0,0,0.1128720566630363,0.04542199894785881,0.9925707578659058,0.717822253704071,0.6869841814041138,-0.1130662634968758,-0.6870161294937134,0.7252514362335205,0.04493638500571251,0.7573306857594695,0.1562293038485165,-0.06254761850625014,0.005143177695572376,-0.09930592775344849,0.01147529296576977,-0.00633104145526886,0.02205187082290649,-0.00571894645690918,0.1476593017578125,-0.687042236328125,-0.6767730712890625,0.1130662634968758,-0.6869841814041138,-0.717822253704071,-0.0534171536564827,0.05082562938332558,-0.07429340481758118,0,0,0]
val=X[0]
inp=np.expand_dims(sam,axis=0)
model_1.predict(inp)

array([[0.00654495, 0.1050915 ]], dtype=float32)

In [19]:
model_arc=load_model('fed_model.h5')
sam=[0,0,0,0.1575399488210678,0.0433773510158062,0.9865594506263733,0.8465654850006104,0.508436918258667,-0.1575399488210678,-0.508436918258667,0.8600059747695923,0.0433773510158062,0.5333685948695585,0.1839884400688954,-0.05039572332323516,0.04226274043321609,-0.08296206593513489,-0.05209726095199585,0.008109480142593384,0.08970516920089722,0.01483941078186035,0.1676483154296875,-0.532867431640625,-0.798065185546875,0.1575399488210678,-0.508436918258667,-0.8465654850006104,-0.6094287037849426,-0.02896853536367416,-0.3641055226325989,0,0,0]

val_1=X[320]
inp_1=np.expand_dims(sam,axis=0)
model_arc.predict(inp_1)

array([[0.3577438 , 0.25165567]], dtype=float32)

In [11]:
import coremltools

In [13]:
coreml_model = coremltools.converters.keras.convert('nudges_dev__1_new.h5',class_labels = ['no','yes'])
coreml_model.save('nudges_dev__1_new.mlmodel')

0 : input_1, <keras.engine.input_layer.InputLayer object at 0x1471778d0>
1 : dense_1, <keras.layers.core.Dense object at 0x147177940>
2 : dense_1__activation__, <keras.layers.core.Activation object at 0x14ae9cd68>
3 : dense_2, <keras.layers.core.Dense object at 0x1471779b0>
4 : dense_2__activation__, <keras.layers.core.Activation object at 0x14ae9c4e0>
